Выводы:

1) Таблица train_job_satisfaction_rate_df:
    - длительность работы в компании: от 1 до 8 лет, большинство сотрудников - 1 год. Половина сотрудников - от 2 до 6 лет. Выбросов нет
    - оценка качества работы сотрудника: чаще всего на 4 балла, половину сотрудников оценивают на 3-4 балла. Есть группа сотрудников, которых оценили на 1 балл - их 5% (считаются выбросом)

    - зарплата: нужно оценивать относительно уровня сотрудника. Общий разброс - от 12 до 100 тыс. Основная масса получает от 22 до 45 тыс
    - <font color='red'>уровень удовлетворенности сотрудника: основная масса лежит от 0.38 до 0.7. Самая частая оценка - около 0.68 </font>
    - разнос по департаментаментам:  продажи/ технологии/закупки/маркетинг/кадры = 3:2:1:1 соотношение
    - уровни сотрудников: джун/мидл/сениор = 5/5/1 соотношение
    - загрузка сотрудника: низкая/средняя/высокая = 1,5/3/1 соотношение
    - повысили в прошлом году- подавляющее меньшинство (3%)
    - нарушения за последний год  - у 15% 

2) Таблица test_features_df: анализ показал те же параметры, что и у тренировочной выборки

3) Таблица test_target_job_satisfaction_rate_df: анализ показал те же параметры, что у целевого признака в тренировочной выборке

In [ ]:
def plot_scatter_for_df(some_df,feat1, feat2, target_feature):

    # Создаем scatter-plot с линиями тренда для каждой группы точек
    fig = px.scatter(some_df, x=feat1, y=feat2, color= target_feature, #symbol=target_feature,
                    title= f'График рассеяния  {feat1} по {feat2} с учетом признака "{target_feature}"',
                    labels={feat1: feat1.capitalize(), feat2: feat2.capitalize()},
                    color_discrete_map={'Снизилась': 'blue', 'Прежний уровень': 'yellow'},  # Цвета точек для двух значений target_feat
    #                symbol_map={'Снизилась': 'circle', 'Прежний уровень': 'square'})  # Формы точек для двух значений target_feat
                    )


    # Фильтруем данные для каждой группы точек
    data_yellow = some_df[some_df[target_feature] == 'Прежний уровень']
    data_blue = some_df[some_df[target_feature] == 'Снизилась']

    # Добавляем линию тренда для красных точек
    fig.add_trace(px.scatter(data_yellow, 
                            x=feat1, 
                            y=feat2, 
                            trendline='ols',
                            trendline_color_override= 'orange',
                            color_continuous_scale=False).data[1])

    # Добавляем линию тренда для синих точек
    fig.add_trace(px.scatter(data_blue, 
                            x=feat1, 
                            y=feat2, 
                            trendline='ols',
                            trendline_color_override= 'blue',
                            color_continuous_scale=False).data[1])


    # Добавляем описание легенды

    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', marker=dict(color='orange'), 
                                showlegend=True, name=f'линия тренда для {target_feature} = Прежний уровень'))

    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', marker=dict(color='blue'), 
                                showlegend=True, name=f'линия тренда для {target_feature} = Снизилась'))

    # Перемещаем легенду над графиком
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))


    # Показываем график
    fig.show()


In [ ]:
plot_scatter_for_df(some_df= df_market,
                    feat1='last_last_rev',
                    feat2='current_dur_minutes', 
                    target_feature = 'consum_activity')

In [ ]:
target_feature = 'consum_activity'
list_of_columns = df_market.drop(['id', target_feature], axis=1)

list_of_err = []

for pair_of_cols in list_of_pairs:
    feat1= pair_of_cols[0]
    feat2 = pair_of_cols[1]

    if feat1 in  ['service_type', 'is_allow_report', 'pop_cat_goods'] or feat2 in  ['service_type', 'is_allow_report', 'pop_cat_goods']:
        continue
    try:
        plot_scatter_for_df(some_df= df_market,
                feat1=feat1,
                feat2=feat2, 
                target_feature = target_feature)

    except:
        list_of_err.append(pair_of_cols)
        continue



In [ ]:
from itertools import combinations

list_of_pairs = list(combinations(df_market.drop(['id', 'consum_activity'], axis=1).columns,
                                r=2))

list_of_pairs

In [ ]:
def plot_scatter_new(some_df, feat1, feat2, target_feature):

        # Разделяем данные на две группы в соответствии со значением target_feature
    good_data = some_df[some_df[target_feature] == 'Прежний уровень']
    bad_data = some_df[some_df[target_feature] == 'Снизилась']

    # Создаем scatter-plot
    plt.figure(figsize=(12, 4))

    # Создаем линии тренда для обеих групп точек
    def plot_trendline(data, color, label):
        x = data[feat1]
        y = data[feat2]
        A = np.vstack([x, np.ones(len(x))]).T
        m, c = np.linalg.lstsq(A, y, rcond=None)[0]
        plt.plot(x, m*x + c, color=color, linestyle='-', label=label)

    

    # Отображаем точки для обеих групп
    plt.scatter(bad_data[feat1], bad_data[feat2], color='tomato', label='Снизилась')
    plt.scatter(good_data[feat1], good_data[feat2], color='blueviolet', label="Прежний уровень")

    # Отображаем трендлинии для обеих групп
    plot_trendline(good_data, 'blue', 'Тренд-линия "Прежний уровень"')
    plot_trendline(bad_data, 'red', 'Трендлиния "Снизилась"')

    # Добавляем метки и заголовки
    plt.xlabel(f'{feat1}')
    plt.ylabel(f'{feat2}')
    plt.title(f'Scatter Plot для {feat1} по {feat2}')

    # Добавляем легенду
    plt.legend()

    # Показываем график
    plt.grid(True)
    plt.show()